In [ ]:
import os
os.chdir('/mmsegmentation')

In [ ]:
import time
import torch
from mmengine.config import Config
from mmseg.utils import register_all_modules
from mmseg.registry import MODELS, DATASETS
from mmengine.runner import load_checkpoint
from mmengine.dataset import default_collate

# LOAD CONFIGURATION AND REGISTER MODULES
cfg_path = '/configs/model_config.py'
cfg = Config.fromfile(cfg_path)
register_all_modules()

# BUILD MODEL
model = MODELS.build(cfg.model)
model.cuda()
model.eval()

# LOAD CHECKPOINT
checkpoint_path = 'work_dirs/best_checkpoint.pth'
load_checkpoint(model, checkpoint_path, map_location='cuda')

# LOAD VALIDATION DATASET
val_dataset_cfg = cfg.val_dataloader['dataset']
val_dataset = DATASETS.build(val_dataset_cfg)

# MEASUREMENT SETTINGS
n_repeats = 10
total_imgs = len(val_dataset) * n_repeats
print(f"Evaluating {len(val_dataset)} images × {n_repeats} repetitions = {total_imgs} inferences")

# MEASURE INFERENCE SPEED
total_time = 0.0
with torch.no_grad():
    for r in range(n_repeats):
        for i in range(len(val_dataset)):
            data = val_dataset[i]
            batch = default_collate([data])
            batch = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
            batch['data_samples'] = [ds.to('cuda') for ds in batch['data_samples']]

            start = time.time()
            _ = model.test_step(batch)
            end = time.time()
            total_time += (end - start)

# RESULTS
avg_time = total_time / total_imgs
fps = 1.0 / avg_time

print(f"\n INFERENCE COMPLETED")
print(f"Average time per image: {avg_time:.4f} seconds")
print(f"Average FPS: {fps:.2f} frames/second")